In [1]:
import numpy as np
import matplotlib.pyplot as plt
from lib import sig_gen
import torch
import torch.nn as nn
from lib import sig_gen
# import torch.optim
torch.__version__

'1.0.1.post2'

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.version.cuda 
# torch.backends.cudnn.version()  # Corresponding cuDNN version

'9.0.176'

In [13]:
symb, sps = 64, 4
mi, bt = 0.5, 0.3
msg = np.random.randint(0, 2, symb)
sig = sig_gen.gfsk_mod(msg, sps, bt, mi)

sig.shape

(256,)

In [2]:
sig2 = np.vstack((sig, sig, sig))
sig2 = np.expand_dims(sig2, 1)[:, :, 0:-1: 4]

NameError: name 'sig' is not defined

In [ ]:
class CVcnn(torch.nn.Module):
    def __init__(self):
        super(CVcnn, self).__init__()
        
        self.conv1_r = nn.Conv1d(1, 8, 5)
        self.conv1_i = nn.Conv1d(1, 8, 5)
        
        self.conv2_r = nn.Conv1d(8, 8, 5)
        self.conv2_i = nn.Conv1d(8, 8, 5)
        
    def forward(self, x_r, x_i):
        (y_r, y_i) = (self.conv1_r(x_r) - self.conv1_i(x_i),
                      self.conv1_r(x_i) + self.conv1_i(x_r))
        
        (y_r, y_i) = (self.conv2_r(y_r) - self.conv2_i(y_i),
                      self.conv2_r(y_i) + self.conv2_i(y_r))
        
        return y_r, y_i
    
    
net = CVcnn()
print('Number of parameters: {}'.format(sum(p.numel() for p in net.parameters())))

x_r = torch.from_numpy(sig2.real).float()
x_i = torch.from_numpy(sig2.imag).float()
y_r, y_i = net(x_r, x_i)

print(x_r.shape, y_r.shape)

In [ ]:
class RVcnn(torch.nn.Module):
    def __init__(self):
        super(RVcnn, self).__init__()
        
        self.conv1 = nn.Conv1d(2, 8, 5)
        self.conv2 = nn.Conv1d(8, 8, 5)
#         self.conv1_i = nn.Conv1d(1, 2, 5) 
    def forward(self, x):
        y = self.conv1(x)
        y = self.conv2(y)
#         y_r = self.conv1_r(x_r) - self.conv1_i(x_i)
#         y_i = self.conv1_r(x_i) + self.conv1_i(x_r)
        return y

net = RVcnn()
print('Number of parameters: {}'.format(sum(p.numel() for p in net.parameters())))

x_r = torch.from_numpy(sig2.real).float()
x_i = torch.from_numpy(sig2.imag).float()
x = torch.cat((x_r, x_i), 1)
y= net(x)

print(x.shape, y.shape)